In [1]:
import numpy as np
#import scipy.misc
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline

import math
import random

#from random import shuffle

In [2]:
X = [1,2,3,4,5]
random.shuffle(X)
X

[1, 4, 3, 5, 2]

In [3]:
numNetworks = 100
numNodesPerNetwork = 100
inputFeatures = 10
outputFeatures = 5

globalInnovationNumber = 0

assert inputFeatures < numNodesPerNetwork
assert outputFeatures < numNodesPerNetwork

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def clamp(x, a, b):
    return min(max(a, x), b)

class Node:
    """
    This represents a neural network node with an integer label nodeId, 
    and a special type, one of ["input", "output", "hidden"]
    """
    def __init__(self, nodeId, nodeType):
        self.nodeId = nodeId
        assert nodeType == "input" or nodeType == "output" or nodeType == "hidden"
        self.nodeType = nodeType
        
    def copy(self):
        return Node(self.nodeId, self.nodeType)    
        
    def __str__(self):
        return "Node: " + str(self.nodeId) + ", " + self.nodeType
    
        
class Edge:
    """
    This class represent a connection (axon) in a neuroevolution network with its data.
    nodeIn and nodeOut are the ids of the input and output nodes,
    enabled is a boolean representing its ability to propogate into future neural networks,
    innovId is the labeled innovation number which corresponds to the same edge in other NNs,
    and weight is the normal edge weight.
    """
    def __init__(self, nodeIn, nodeOut, enabled, innovId, weight):
        self.nodeInId = nodeIn
        self.nodeOutId = nodeOut
        self.enabled = enabled
        self.innovId = innovId
        self.weight = weight
        
    def copy(self):
        return Edge(self.nodeInId, self.nodeOutId, self.enabled, self.innovId, self.weight)
        
    def __str__(self):
        return "Edge: " + str(self.nodeInId) + " -> " + str(self.nodeOutId) + ", Enabled: " + \
            str(self.enabled) + ", InnovID: " + str(self.innovId) + ", Weight: " + str(self.weight)
    
class NeuroevolutionNetwork:
    
    # self.nodes is a dictionary indexed by (node id -> node object)
    # self.edges is indexed by (in node id -> edge object, from in node to out node)
    def __init__(self, numNodes, numInput, numOutput):
        assert numInput + numOutput < numNodes
        
        self.nodes = dict() 
        self.edges = dict()
        self.listEdges = []
        self.numNodesCounter = 0
        
        self.numInput = numInput
        self.numOutput = numOutput
        self.initNodes = numNodes
        
        for _ in range(numInput):
            self.addNode("input")
        for _ in range(numOutput):
            self.addNode("output")
        for _ in range(numNodes - numInput - numOutput):
            self.addNode("hidden")
        
    # Special copy method to handle it ourselves, so it's not just a shallow/deep clone
    def copy(self):
        copyNetwork = NeuroevolutionNetwork(self.initNodes, self.numInput, self.numOutput)
        
        for nodeId, node in self.nodes.items():
            # For every node id, copy its node to the new network
            copyNetwork.nodes[nodeId] = node.copy()
            
        for nodeId, edgeList in self.edges.items():
            # For every node in id, copy the list of outgoing edges to the new network
            copyEdgeArr = []
            for edge in edgeList:
                copyEdgeArr.append(edge.copy())
            copyNetwork.edges[nodeId] = copyEdgeArr  
            
        for pair in self.listEdges:
            copyNetwork.listEdges.append(tuple(pair))
            
        return copyNetwork
    
    def printAllEdges(self):
        for nodeInId, nodeArr in self.edges.items():
            print(nodeInId)
            for node in nodeArr:
                print(str(node))
                
    def numNodes(self):
        return len(self.nodes)
    
    def numEdges(self):
        return len(self.listEdges)
        
    def addNode(self, nodeType):
        nodeId = self.numNodesCounter
        self.nodes[nodeId] = Node(nodeId, nodeType)
        self.numNodesCounter += 1
        return nodeId
        
    def hasNode(self, nodeId):
        return nodeId in self.nodes    
        
    # Return true if a connection exists between nodes with these ids,
    # a connection from in -> out
    def isConnected(self, nodeInId, nodeOutId):    
        """if hasNode(nodeInId) and hasNode(nodeOutId):
            if nodeInId in self.edges:
                connectionsOut = self.edges[nodeInId]
                for nodeOut in connectionsOut:
                    if nodeOut.nodeId == nodeOutId:
                        return True
        return False"""
        return (nodeInId, nodeOutId) in self.listEdges
    
    # Return true if there is a valid connection from in -> out
    # Invariant to whether or not the nodes are connected or not
    def allowedConnection(self, nodeInId, nodeOutId):
        if self.hasNode(nodeInId) and self.hasNode(nodeOutId):
            inNode = self.nodes[nodeInId]
            outNode = self.nodes[nodeOutId]
            doubleInput = inNode.nodeType == "input" and outNode.nodeType == "output"
            doubleOutput = inNode.nodeType == "input" and outNode.nodeType == "output"
            return not doubleInput and not doubleOutput
        else:
            return False
        
    # Variant to whether nodes are connected. To be able to add a new connection, the nodes
    # must follow these conditions:
    """
    Nodes exist
    Nodes are not connected in either direction
    The end node is not an input
    The start node is not an output
    New: the new edge cannot create a cycle
    """
    def canAddConnection(self, nodeInId, nodeOutId):
        assert nodeInId != nodeOutId
        if not self.allowedConnection(nodeInId, nodeOutId):
            return False
        inToOutC = self.isConnected(nodeInId, nodeOutId)
        outToInC = self.isConnected(nodeOutId, nodeInId)
        endInput = self.nodes[nodeOutId].nodeType == "input"
        startOutput = self.nodes[nodeInId].nodeType == "output"
        longPathExists = self.findPathBetween(nodeOutId, nodeInId) # Check for possible cycles
        return not (inToOutC or outToInC or endInput or startOutput or longPathExists)
        
    # Select a random pair of nodes, that have no connection either way,
    # and are not both input and not both output,
    # and add a connection between them, from in -> out.
    def mutateAddConnection(self):
        assert len(self.nodes) >= 2
        
        # Generate a shuffled list of possible future connections
        availIds = [node.nodeId for _,node in self.nodes.items()]
        lenIds = len(availIds)
        availablePairs = [(availIds[i], availIds[j]) for i in range(lenIds) for j in range(lenIds)]
        random.shuffle(availablePairs)
        
        # Find two nodes that have no connection
        # Once found, add the one new connection, and return
        while True: 
            if len(availablePairs) == 0: # No more possible connections to check
                return
            pair = availablePairs.pop() 
            if pair[0] == pair[1]:
                continue
            # Select a random connection, that is neither a self connection (does not exist),
            # nor an existing connection in either direction.
            # inToOutC = self.isConnected(pair[0], pair[1])
            # outToInC = self.isConnected(pair[1], pair[0])
            if self.canAddConnection(pair[0], pair[1]):
                global globalInnovationNumber
                self.addEdgeWithInnovId(pair[0], pair[1], True, globalInnovationNumber, edgeWeight=random.random())
                globalInnovationNumber += 1
                return
            
    def mutateRemoveConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        nodeInId, nodeOutId = None, None
        candidateEdges = [i for i in range(len(self.listEdges))]
        random.shuffle(candidateEdges)
        while True:
            if len(candidateEdges) == 0:
                return
            randomEdgeIndex = candidateEdges.pop(0)
             
            oldEdge = tuple(self.listEdges[randomEdgeIndex])
            nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
            if self.allowedConnection(nodeInId, nodeOutId):
                break
            else:
                continue

        self.removeEdge(nodeInId, nodeOutId)
            
    """
    This mutates the graph by finding a random existing edge,
    and then splitting into two edges with a new connection in the middle,
    as per Stanley & Miikkulainen.
    """    
    def mutateSplitConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        nodeInId, nodeOutId = None, None
        candidateEdges = [i for i in range(len(self.listEdges))]
        random.shuffle(candidateEdges)
        edgeObj = None
        while True:
            if len(candidateEdges) == 0:
                return
            randomEdgeIndex = candidateEdges.pop(0)
             
            oldEdge = tuple(self.listEdges[randomEdgeIndex])
            nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
            
            edgeObj = self.findEdge(nodeInId, nodeOutId)
            if not edgeObj.enabled:
                continue
            
            if self.canAddConnection(nodeInId, nodeOutId):
                break
            else:
                continue
        
        # TEMP:
        # self.removeEdge(nodeInId, nodeOutId)
        # No, do not remove the edge. It must be preserved to play a part in later
        # NN "reproduction" and similarity measures

        edgeObj.enabled = False
        
        newMiddleNode = self.addNode("hidden")
        
        global globalInnovationNumber
        self.addEdgeWithInnovId(nodeInId, newMiddleNode, True, globalInnovationNumber, edgeWeight=random.random())
        globalInnovationNumber += 1
        self.addEdgeWithInnovId(newMiddleNode, nodeOutId, True, globalInnovationNumber, edgeWeight=random.random())
        globalInnovationNumber += 1
            
        
    def mutateChangeWeight(self):        
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        randomEdgeIndex = int(len(self.listEdges) * random.random())
        oldEdge = tuple(self.listEdges[randomEdgeIndex])
        nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
        edgeObj = self.findEdge(nodeInId, nodeOutId)
        
        weight = edgeObj.weight
        if random.random() < 0.5:
            weight -= 0.1
        else:
            weight += 0.1
        weight = clamp(weight, 0, 1)
        
        edgeObj.weight = weight
    
    
    def findEdge(self, nodeInId, nodeOutId):
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            possibleEdge = self.edges[nodeInId][nodeOutIndex]
            if possibleEdge.nodeOutId == nodeOutId:
                return possibleEdge
        return None
        
    def removeEdge(self, nodeInId, nodeOutId):
        assert nodeInId in self.edges
        assert (nodeInId, nodeOutId) in self.listEdges
        
        self.listEdges.remove((nodeInId, nodeOutId))
        
        # Look for the correct node within the outgoing connections
        # i.e. find the connection in -> out
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            nodeOut = self.edges[nodeInId][nodeOutIndex]
            if nodeOut.nodeOutId == nodeOutId:
                self.edges[nodeInId].pop(nodeOutIndex)
                return  
            
    # Simple tree traversal, to check for a path between in -> out, of any length
    def findPathBetween(self, nodeInId, nodeOutId):
        fringe = [nodeInId]
        marked = set()
        while len(fringe) > 0:
            v = fringe.pop(0)
            if v == nodeOutId: # This case is if nodeInId == nodeOutId
                return True
            marked.add(v)
            if v not in self.edges: # No edges starting from v
                continue
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if w == nodeOutId:
                    return True
                if w not in marked:
                    fringe.append(w)
        return False
            
    def addEdgeWithInnovId(self, nodeInId, nodeOutId, enabled, innovId, edgeWeight):
        assert self.canAddConnection(nodeInId, nodeOutId)
        
        newEdge = Edge(nodeInId, nodeOutId, enabled, innovId, edgeWeight)
        
        self.listEdges.append((nodeInId, nodeOutId))
        if nodeInId not in self.edges:
            self.edges[nodeInId] = []
        self.edges[nodeInId].append(newEdge)
            
        
    """
    Get all of the directed edges of the NN, but in reverse i.e. incoming edges,
    also representing an in-degree count indexed by vertex.
    """
    def getReversedEdges(self):
        reversedEdges = dict()
        
        for edgeInId, edgeList in self.edges.items():
            for edge in edgeList:
                if edge.nodeOutId not in reversedEdges:
                    reversedEdges[edge.nodeOutId] = []
                reversedEdges[edge.nodeOutId].append(edge.nodeInId)
        
        return reversedEdges
    
    """
    The new forwarding algorithm works as follows, in a DAG:
    obtain the edges of the graph, reversed;
    determine the in-degree of each vertex, using above;
    use Kahn's algorithm -> remove vertices of in-degree zero,
    repeat until end or a valid topological sort has been found;
    update vertices in topological order
    
    Proof that Kahn's algorithm terminates in a valid topo sort <-> graph is a DAG
    is left as an exercise for the reader.
    """
    
    """
    Return a list of the nodes 
    that represent a valid topological sort,
    starting with input edges (guaranteed to be in-degree zero).
    """
    def graphTopoSort(self):
        reversedEdges = self.getReversedEdges()
        
        marked = dict()
        topoSort = []
        
        for nodeId,_ in self.nodes.items():
            marked[nodeId] = False
        
        while len(topoSort) < len(self.nodes):
            degZeroIncomingIds = []
            
            for nodeId,_ in self.nodes.items():
                # Boolean short circ. V
                # The second case can only come about when connections are discarded
                if (nodeId not in reversedEdges or len(reversedEdges[nodeId]) == 0) and not marked[nodeId]: 
                    # nodeId must have an in-degree zero
                    degZeroIncomingIds.append(nodeId)
                    marked[nodeId] = True
                    
                    # No outgoing edges, we're done
                    if nodeId not in self.edges:
                        continue
                    
                    # Discard all outgoing connections
                    outgoingEdges = self.edges[nodeId]
                    
                    for edge in outgoingEdges:
                        if edge.nodeOutId in reversedEdges:
                            reversedEdges[edge.nodeOutId].remove(nodeId)
            
            topoSort = topoSort + degZeroIncomingIds
        return topoSort
    
    """
    Calculate the entire NN output, starting from the input.
    Again, the topological ordering is used to guarantee the nodes are calculated correctly
    i.e. all incoming edges have valid values.
    """
    def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        # nodeData is the final result of the NN, indexed by nodeId
        # Initialize the input, which is ideally of shape (input nodes X 1)
        nodeData = dict()
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
        
        # Update the nodes in topo order, using the incoming edges to determine weights and values
        reversedEdges = self.getReversedEdges()
        topoSort = self.graphTopoSort()
        
        for nodeId in topoSort:
            nodeType = self.nodes[nodeId].nodeType
            
            # Input nodes must not be overridden. Their degree is technically zero
            if nodeType == 'input':
                continue
            
            # Find the list of incoming edges
            incomingEdges = []
            if nodeId in reversedEdges:
                incomingEdges = reversedEdges[nodeId]
            
            result = 0
            if len(incomingEdges) == 0:
                result = 0.5
            else:
                nodeSum = 0
                # Sum up the incoming values, and take the activation function of the sum,
                # while noting that only enabled edges count in the calculation.
                for inVertex in incomingEdges:
                    inEdge = self.findEdge(inVertex, nodeId)
                    if inEdge.enabled:  
                        value = nodeData[inVertex]
                        weight = inEdge.weight
                        nodeSum += value * weight
                result = sigmoid(nodeSum)
                
            nodeData[nodeId] = result
        
        return nodeData
    
    def score(self, currentNodeData, desiredOutput, lossFunction):
        predictedOutput = np.zeros((self.numOutput))
        for i in range(0, self.numOutput):
            predictedOutput[i] = currentNodeData[self.numInput + i]
        assert predictedOutput.shape == desiredOutput.shape
        loss = lossFunction(predictedOutput, desiredOutput)
        return predictedOutput, desiredOutput, loss
        
    """def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
            
        # Use Dijkstra's algorithm to update the NN as we go, 
        # using the input data as the starting fringe.
        
        # Ideally, the NN is a DAG, but this will work even if not.
        
        nodeData = dict()
        prev = dict()
        marked = dict()
        
        queue = [i for i in range(len(self.numInput))]
        
        for nodeId, _ in self.nodes.items():
            nodeData[nodeId] = -9999
            prev[nodeId] = -1
            marked[nodeId] = False
        
        while len(queue) > 0:
            v = queue.pop() # A node id
            marked[v] = True
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if :
                    nodeData[]"""

In [4]:
# Unit tests for neuroevolution network

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)

# Nodes are initialized correctly
"""
for k,v in testNet.nodes.items():
    print(v)
"""
    
# Edges are initialized correctly
print("-----------------------------------")
print("Test #2")
print(testNet.edges, testNet.listEdges)

print("-----------------------------------")
print("Test #3")
for _ in range(5):
    testNet.mutateAddConnection()

testNet.printAllEdges()
    
print(testNet.listEdges)

print("-----------------------------------")
print("Test #4")

testEdge = testNet.listEdges[0]
testNet.removeEdge(testEdge[0], testEdge[1])

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5")

testNet.mutateSplitConnection()

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5.5")

for _ in range(10):
    testNet.mutateChangeWeight()

testNet.printAllEdges()

print("-----------------------------------")
print("Test #6")

testNetCopy = testNet.copy()
testNetCopy.mutateAddConnection()

print("Original network: ")
testNet.printAllEdges()
print("Copy network: ")
testNetCopy.printAllEdges()

-----------------------------------
Test #2
{} []
-----------------------------------
Test #3
23
Edge: 23 -> 48, Enabled: True, InnovID: 0, Weight: 0.8281202645143207
85
Edge: 85 -> 71, Enabled: True, InnovID: 1, Weight: 0.025184255205872863
39
Edge: 39 -> 55, Enabled: True, InnovID: 2, Weight: 0.8708223769831785
94
Edge: 94 -> 64, Enabled: True, InnovID: 3, Weight: 0.23991237150287614
44
Edge: 44 -> 68, Enabled: True, InnovID: 4, Weight: 0.09568290520991818
[(23, 48), (85, 71), (39, 55), (94, 64), (44, 68)]
-----------------------------------
Test #4
23
85
Edge: 85 -> 71, Enabled: True, InnovID: 1, Weight: 0.025184255205872863
39
Edge: 39 -> 55, Enabled: True, InnovID: 2, Weight: 0.8708223769831785
94
Edge: 94 -> 64, Enabled: True, InnovID: 3, Weight: 0.23991237150287614
44
Edge: 44 -> 68, Enabled: True, InnovID: 4, Weight: 0.09568290520991818
[(85, 71), (39, 55), (94, 64), (44, 68)]
-----------------------------------
Test #5
23
85
Edge: 85 -> 71, Enabled: True, InnovID: 1, Weight: 0

In [5]:
testNode = Node(85, 'hidden')
print(testNode.copy())

Node: 85, hidden


In [6]:
"""
Calculate the distance delta between two neuroevolution networks.
This heuristic is important in determining a normalized fitness score,
which rewards successful organisms but ensures that one species does not take over.

In Stanley, Miikkulainen, this is defined as

delta = c_1 * E / N + c_2 * D / N + c_3 * W
where E is the number of excess genes,
D is the number of disjoint genes, 
N is the normalization factor (number of genes total),
and W is the average weight distance between shared genes.

Ideally, the weights have constraint 0 <= c_1, c_2, c_3 <= 1, 
and should also be consistent.
"""
def neuroNetworkDiff(nn1, nn2, c1, c2, c3):
    # Calculate excess number of genes by count
    excess = abs(nn1.numEdges() - nn2.numEdges())
    
    # Disjoint genes just need to be counted,
    # shared genes must have their weight differences averaged
    disjoint = 0
    sharedGenesDiff = []
    
    numGenes = max(nn1.numEdges(), nn2.numEdges())
    
    firstEdgeInnovIds = set()
    weightsByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            weightsByInnovId[edge.innovId] = edge.weight
    
    for _,outEdges in nn2.edges.items():
        for secondEdge in outEdges:
            if secondEdge.innovId in firstEdgeInnovIds:
                firstEdgeWeight = weightsByInnovId[secondEdge.innovId]
                secondEdgeWeight = secondEdge.weight
                # secondEdge = nn2.edges[node.nodeId]
                sharedGenesDiff.append(abs(firstEdgeWeight - secondEdgeWeight))
            else:
                disjoint += 1

    avgDiffW = np.sum(sharedGenesDiff) / len(sharedGenesDiff)
    
    # print(excess, disjoint, numGenes, avgDiffW)
    
    return c1 * excess / numGenes + \
        c2 * disjoint / numGenes + \
        c3 * avgDiffW
        

def crossNeuralNetworks(nn1, nn2):
    assert nn1.numInput == nn2.numInput
    assert nn1.numOutput == nn2.numOutput
    newNumNodes = max(nn1.numNodes(), nn2.numNodes())
    
    crossedNet = NeuroevolutionNetwork(newNumNodes, nn1.numInput, nn1.numOutput)
    
    firstEdgeInnovIds = set()
    secondEdgeInnovIds = set()
    firstEdgesByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    secondEdgesByInnovId = dict()
    enabledByInnovId = dict()
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            firstEdgesByInnovId[edge.innovId] = edge
            enabledByInnovId[edge.innovId] = edge.enabled
    
    for _,outEdges in nn2.edges.items():
        for edge in outEdges:
            secondEdgeInnovIds.add(edge.innovId)
            secondEdgesByInnovId[edge.innovId] = edge
            if edge.innovId in enabledByInnovId:
                enabledByInnovId[edge.innovId] = enabledByInnovId[edge.innovId] and edge.enabled
            else:
                enabledByInnovId[edge.innovId] = edge.enabled
    
    """
    As per Stanley and Miikkulainen, merge the edges together by their innovation ids.
    Disjoint/excess innovations transfer fully, shared innovations must merge,
    weight by average, enabled by AND
    """
    
    sharedInnov = firstEdgeInnovIds.intersection(secondEdgeInnovIds)
    onlyFirstInnov = firstEdgeInnovIds.difference(secondEdgeInnovIds)
    onlySecondInnov = secondEdgeInnovIds.difference(firstEdgeInnovIds)
    
    #nodeIn, nodeOut, enabled, innovId, weight)
    for innovId in onlyFirstInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
        
    for innovId in onlySecondInnov:
        baseEdge = secondEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
            
    for innovId in sharedInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        
        firstWeight = firstEdgesByInnovId[innovId].weight
        secondWeight = secondEdgesByInnovId[innovId].weight
        avgWeight = 0.5 * (firstWeight + secondWeight)
        
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, avgWeight)
    
    return crossedNet
                

def squaredLoss(predicted, desired):
    return np.sum((predicted - desired) ** 2)    



In [7]:
print("-----------------------------------")
print("Test #7")

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)
for _ in range(10):
    testNet.mutateAddConnection()
    
testNetMutate = testNet.copy()

for _ in range(2):
    testNetMutate.mutateAddConnection()
for _ in range(5):
    testNetMutate.mutateSplitConnection()
    
testNet.printAllEdges()
testNetMutate.printAllEdges()

print(neuroNetworkDiff(testNet, testNetMutate, 0.5, 0.25, 0.2))

print("-----------------------------------")
print("Test #8")

testEdges = testNet.getReversedEdges()
for nodeId,listEdges in testEdges.items():
    print(nodeId)
    print(listEdges)

print("-----------------------------------")
print("Test #9")

testNet.printAllEdges()
topoSort = testNet.graphTopoSort()
print(topoSort)

-----------------------------------
Test #7
84
Edge: 84 -> 49, Enabled: True, InnovID: 6, Weight: 0.04631645599047418
44
Edge: 44 -> 39, Enabled: True, InnovID: 7, Weight: 0.31122914602529606
24
Edge: 24 -> 42, Enabled: True, InnovID: 8, Weight: 0.3500014355558735
40
Edge: 40 -> 75, Enabled: True, InnovID: 9, Weight: 0.853306091690171
57
Edge: 57 -> 44, Enabled: True, InnovID: 10, Weight: 0.8787030691642096
51
Edge: 51 -> 65, Enabled: True, InnovID: 11, Weight: 0.749766203956481
91
Edge: 91 -> 79, Enabled: True, InnovID: 12, Weight: 0.02220492990536349
85
Edge: 85 -> 93, Enabled: True, InnovID: 13, Weight: 0.24113042118055472
3
Edge: 3 -> 21, Enabled: True, InnovID: 14, Weight: 0.612454853822975
50
Edge: 50 -> 33, Enabled: True, InnovID: 15, Weight: 0.266495368389429
84
Edge: 84 -> 49, Enabled: True, InnovID: 6, Weight: 0.04631645599047418
44
Edge: 44 -> 39, Enabled: True, InnovID: 7, Weight: 0.31122914602529606
24
Edge: 24 -> 42, Enabled: True, InnovID: 8, Weight: 0.3500014355558735
4

In [8]:
print("-----------------------------------")
print("Test #10")

testNet = NeuroevolutionNetwork(15, 5, 5)
for _ in range(20):
    testNet.mutateAddConnection()
for _ in range(3):
    testNet.mutateSplitConnection()
for _ in range(20):
    testNet.mutateAddConnection()
    
testNet.printAllEdges()

-----------------------------------
Test #10
12
Edge: 12 -> 6, Enabled: True, InnovID: 18, Weight: 0.43603360666630353
Edge: 12 -> 14, Enabled: True, InnovID: 22, Weight: 0.08610693632931188
Edge: 12 -> 13, Enabled: True, InnovID: 32, Weight: 0.27121428072875897
Edge: 12 -> 5, Enabled: True, InnovID: 34, Weight: 0.9153243603836091
Edge: 12 -> 9, Enabled: True, InnovID: 37, Weight: 0.665378729842033
Edge: 12 -> 10, Enabled: True, InnovID: 44, Weight: 0.6002100132166992
Edge: 12 -> 8, Enabled: True, InnovID: 48, Weight: 0.9969284312139858
1
Edge: 1 -> 11, Enabled: True, InnovID: 19, Weight: 0.5372144714096947
Edge: 1 -> 12, Enabled: True, InnovID: 40, Weight: 0.7090194007852667
Edge: 1 -> 14, Enabled: True, InnovID: 51, Weight: 0.16358128535455263
13
Edge: 13 -> 5, Enabled: True, InnovID: 20, Weight: 0.11069690068913807
Edge: 13 -> 11, Enabled: True, InnovID: 23, Weight: 0.34825190781597704
Edge: 13 -> 10, Enabled: True, InnovID: 28, Weight: 0.510619090938541
Edge: 13 -> 7, Enabled: True

In [9]:
testNet.forwardStep([1.0, 0.0, 1.0, 0.0, 1.0])
# [16, 10, 4, 1, 97]
# -> [4,3.2,2,1,9.7]

{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.82709418908741561,
 6: 0.79380340944639027,
 7: 0.82549962268512123,
 8: 0.82182270781074085,
 9: 0.71665815844268776,
 10: 0.80511428492528281,
 11: 0.76907459117430765,
 12: 0.53469943748972593,
 13: 0.81051196278640913,
 14: 0.61429851295244964}

In [10]:
print("-----------------------------------")
print("Test #11")

# score(self, currentNodeData, desiredOutput, lossFunction):

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
totalNodeData = testNet.forwardStep(inputData)
desired = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

predicted, _, testLoss = testNet.score(totalNodeData, desired, squaredLoss)
print(predicted)
print(desired)
print(testLoss)

-----------------------------------
Test #11
[ 0.82709419  0.79380341  0.82549962  0.82182271  0.71665816]
[ 0.  1.  0.  1.  0.]
1.95339752215


In [11]:
print("-----------------------------------")
print("Test #12")

testNet = NeuroevolutionNetwork(15, 5, 5)

for _ in range(5):
    testNet.mutateAddConnection()
    
testNetMutate1 = testNet.copy()
testNetMutate2 = testNet.copy()

for _ in range(5):
    testNetMutate1.mutateAddConnection()
for _ in range(2):
    testNetMutate1.mutateSplitConnection()

for _ in range(5):
    testNetMutate2.mutateAddConnection()
for _ in range(2):
    testNetMutate2.mutateSplitConnection()
    
print("**************************")
testNet.printAllEdges()
print("**************************")
testNetMutate1.printAllEdges()
print("**************************")
testNetMutate2.printAllEdges()
print("**************************")

crossedNet = crossNeuralNetworks(testNetMutate1, testNetMutate2)
crossedNet.printAllEdges()

-----------------------------------
Test #12
**************************
10
Edge: 10 -> 7, Enabled: True, InnovID: 58, Weight: 0.7622812834780038
2
Edge: 2 -> 11, Enabled: True, InnovID: 59, Weight: 0.4827020338955865
11
Edge: 11 -> 13, Enabled: True, InnovID: 60, Weight: 0.7149544678893673
13
Edge: 13 -> 9, Enabled: True, InnovID: 61, Weight: 0.20303788290226488
14
Edge: 14 -> 8, Enabled: True, InnovID: 62, Weight: 0.9243379138596438
**************************
10
Edge: 10 -> 7, Enabled: True, InnovID: 58, Weight: 0.7622812834780038
2
Edge: 2 -> 11, Enabled: True, InnovID: 59, Weight: 0.4827020338955865
11
Edge: 11 -> 13, Enabled: True, InnovID: 60, Weight: 0.7149544678893673
13
Edge: 13 -> 9, Enabled: True, InnovID: 61, Weight: 0.20303788290226488
Edge: 13 -> 5, Enabled: True, InnovID: 63, Weight: 0.5568098267301437
14
Edge: 14 -> 8, Enabled: True, InnovID: 62, Weight: 0.9243379138596438
Edge: 14 -> 5, Enabled: True, InnovID: 67, Weight: 0.11849495199854954
3
Edge: 3 -> 12, Enabled: Tr

In [21]:
"""
Initialize a world class, which contains multiple neural networks and functions to calculate on them 
and evolve them towards a goal.
"""

numNetworks = 200
numTotalNodes = 25
numInput = 5
numOutput = 5

baseMutations = 5
initialMutations = 5
def iterAddMutations():
    return int(random.random() * 5)
def iterSplitMutations():
    return 0
def iterRemoveMutations():
    return int(random.random() * 4)
def iterWeightMutations():
    return int(random.random() * 7)

mutateProportion = 0.75

similarityThreshold = 0.15 # A distance less than this number implies two NNs are the same "species"
similarityPenalty = 0.15 # In Stanley, Miikkulainen, this number is 1, representing dividing by raw count

class World:
    
    def __init__(self, numNetworks, numTotalNodes, numInput, numOutput):
        self.networks = []
        
        baseNet = NeuroevolutionNetwork(numTotalNodes, numInput, numOutput)
        for _ in range(baseMutations):
            baseNet.mutateAddConnection()
            
        for _ in range(numNetworks - 1):
            baseNetMutate = baseNet.copy()
            for _ in range(initialMutations):
                baseNetMutate.mutateAddConnection()
            self.networks.append(baseNetMutate)
        
    def calcDiffNetMatrix(self):
        # Recommended values: 
        c1 = 0.5
        c2 = 0.25
        c3 = 0.2
        
        n = len(self.networks)
        results = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i < j: # Only calculate half of the table since diff(i,j) == diff(j,i)
                    results[i,j] = neuroNetworkDiff(self.networks[i], self.networks[j], c1, c2, c3)
        return results
        
    """
    Run a forward step through every neural network,
    and then rank all neural networks by an adjusted fitness score, defined as
    
    adj_score_i = raw_score_i / sum_j sh(delta(i,j))
    
    where delta(i,j) is the distance between NNs i and j,
    sh(x) is 1 if x > threshold, otherwise 0,
    and the sum is taken over all other NNs in the world.
    """
    def scoreNetworks(self, inputData, desiredData):
        diffMatrix = self.calcDiffNetMatrix()
        
        nnLoss = np.zeros((len(self.networks)))
        for netId in range(len(self.networks)):
            # Compute raw_score_i, using training data and a loss function
            net = self.networks[netId]
            totalNodeData = net.forwardStep(inputData)
            predicted, _, testLoss = testNet.score(totalNodeData, desiredData, squaredLoss)
            
            # Compute sum_j sh(delta(i,j))
            similarNNCount = 1
            for netId2 in range(len(self.networks)):
                if netId == netId2:
                    continue
                distanceNN = None
                if netId < netId2:
                    distanceNN = diffMatrix[netId, netId2]
                else:
                    distanceNN = diffMatrix[netId2, netId]
                if distanceNN < similarityThreshold:
                    similarNNCount += similarityPenalty
            
            nnLoss[netId] = testLoss # / similarNNCount 
            
        return nnLoss
    
    
    def scoreNetworksOnData(self, largeInputData, largeDesiredData):
        assert largeInputData.shape[0] == largeDesiredData.shape[0]
        data_n = largeInputData.shape[0]
        runningScores = np.zeros((len(self.networks)))
        
        for i in range(data_n):
            inputData = largeInputData[i]
            desiredData = largeDesiredData[i]
            indivScores = self.scoreNetworks(inputData, desiredData)
            indivScores /= data_n
            runningScores += indivScores
            
        return runningScores
    
    
    def evolveNetworks(self, largeInputData, largeDesiredData):
        testScores = self.scoreNetworksOnData(inputData, desiredData)
        testBestNNIndex = np.argsort(testScores)
        bestNetworkIds = testBestNNIndex[0:len(self.networks) // 4]
        
        nextNetworks = []
        for i in range(len(bestNetworkIds)):
            mutateChildren = mutateProportion * 4 # In this hardcode, copy 4 children, mutate 2 of them
            for childIndex in range(4):
                netCopy = self.networks[bestNetworkIds[i]].copy()
                if childIndex <= mutateChildren:
                    for __ in range(iterAddMutations()):
                        netCopy.mutateAddConnection()
                    for __ in range(iterSplitMutations()):
                        netCopy.mutateSplitConnection()
                    for __ in range(iterRemoveMutations()):
                        netCopy.mutateRemoveConnection()
                    for __ in range(iterWeightMutations()):
                        netCopy.mutateChangeWeight()
                nextNetworks.append(netCopy)
        self.networks = nextNetworks
        
        return testScores

def getSortedIndexFromScore(scoresArr):
    pass
    

In [22]:
print("-----------------------------------")
print("Test #13")

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
desiredData = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworks(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

-----------------------------------
Test #13
[ 1.31425663  1.26595477  1.2923982   1.29753364  1.30846893  1.26731929
  1.47210031  1.32146904  1.2272602   1.36752957  1.35400939  1.49352992
  1.27869671  1.20906245  1.4669628   1.29753364  1.53975618  1.24268971
  1.35268786  1.37739403  1.29753364  1.26181453  1.35702007  1.41261855
  1.29753364  1.28859961  1.21721009  1.33909765  1.29231904  1.28060818
  1.35077633  1.29408033  1.29373903  1.29753364  1.30800098  1.30219714
  1.28077943  1.29703873  1.29891073  1.39456639  1.38231771  1.4430211
  1.32838344  1.26839054  1.36121384  1.29753364  1.3056487   1.40857731
  1.43497564  1.3190622   1.3113794   1.18007292  1.29753364  1.1711862
  1.22053335  1.39936124  1.39489126  1.34042981  1.37395725  1.37769715
  1.19704023  1.3090217   1.42417887  1.33914514  1.46676771  1.26174387
  1.35192463  1.4290014   1.17745707  1.3859948   1.32343475  1.25735443
  1.23243797  1.25618972  1.29753364  1.29753364  1.19419826  1.29753364
  1.2975

In [23]:
print("-----------------------------------")
print("Test #14")

inputData = np.array([[1.0, 0.0, 1.0, 0.0, 1.0], [0.24, 0.27, 0.39, 0.56, 0.71]])
desiredData = np.array([[0.0, 1.0, 0.0, 1.0, 0.0], [0.76, 0.73, 0.61, 0.44, 0.29]])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworksOnData(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

-----------------------------------
Test #14
[ 0.71515     0.71515     0.71515     0.76422334  0.72513176  0.78719121
  0.72701118  0.71515     0.72399848  0.71515     0.65790513  0.71515
  0.71515     0.68024695  0.73187947  0.67235016  0.72542222  0.69162567
  0.76811766  0.7667167   0.7187629   0.71515     0.81421083  0.6972807
  0.71515     0.67064289  0.72962303  0.69799816  0.79239919  0.71247719
  0.71515     0.67279186  0.71515     0.71515     0.71515     0.71515
  0.71515     0.67275477  0.71515     0.7619577   0.78156134  0.62373965
  0.72505489  0.71515     0.71515     0.78102075  0.69777278  0.72718353
  0.78357717  0.71101937  0.65760876  0.71515     0.71515     0.71515
  0.71515     0.79347337  0.68295863  0.81098313  0.69488537  0.71515
  0.61952166  0.73128076  0.76935813  0.67096658  0.72987356  0.74020122
  0.71515     0.75683459  0.72216537  0.71515     0.74615043  0.71515
  0.78022468  0.75718945  0.64747041  0.71515     0.68863634  0.69117355
  0.77802127  0.645224

In [24]:
print("-----------------------------------")
print("Test #15")

inputData = np.array([
    [1.0, 0.0, 1.0, 0.0, 1.0], 
    [0.24, 0.27, 0.39, 0.56, 0.71],
    [0.0, 1.0, 0.0, 1.0, 0.0]
])
desiredData = np.array([
    [0.0, 1.0, 0.0, 1.0, 0.0], 
    [0.76, 0.73, 0.61, 0.44, 0.29],
    [1.0, 0.0, 1.0, 0.0, 1.0]
])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
scores = None
for iterIndex in range(20 * 10):
    scores = testWorld.evolveNetworks(inputData, desiredData)
    if iterIndex % 20 == 0:
        print("------------------------")
        print(iterIndex)
        print("Average: " + str(np.sum(scores) / numNetworks))
        print(scores)

-----------------------------------
Test #15
------------------------
0
Average: 0.888262007594
[ 0.8925745   0.9037394   0.88700862  0.88773295  0.8868511   0.87778064
  0.88787624  0.87653674  0.88056784  0.91054874  0.89779983  0.88736764
  0.89145934  0.8883223   0.88788761  0.92191686  0.88175927  0.89987105
  0.88406062  0.92684093  0.89235966  0.90131635  0.88732711  0.89097174
  0.89647916  0.88752916  0.88225088  0.91364144  0.94196142  0.88661201
  0.89415859  0.88422104  0.88352371  0.89642273  0.88543386  0.88166614
  0.8812297   0.88203933  0.89969335  0.88532105  0.90417692  0.90804266
  0.8810377   0.88727191  0.9018702   0.8874061   0.8868511   0.88993227
  0.88883319  0.92234052  0.89963169  0.88139027  0.88804211  0.88848672
  0.88769392  0.91442486  0.88251908  0.88134503  0.88881742  0.89301719
  0.90397805  0.89085783  0.88777882  0.8835442   0.88855637  0.89706773
  0.88754552  0.89358325  0.88755717  0.8868511   0.88779859  0.88145235
  0.8852096   0.89089779  0.

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


------------------------
20
Average: 0.834983160996
[ 0.84322558  0.83515734  0.85071419  0.83670939  0.83906553  0.83735883
  0.83800493  0.85313465  0.85518186  0.83772647  0.85505478  0.85926083
  0.84904728  0.85113005  0.84902198  0.85223779  0.84093062  0.83803374
  0.83698357  0.86282976  0.85074849  0.86121107  0.85847273  0.85875513
  0.83886495  0.83798568  0.83936371  0.83656566  0.84961848  0.83803374
  0.86348896  0.85103323  0.83983618  0.8387454   0.83996939  0.8387454
  0.87656378  0.84047683  0.85397148  0.89895085  0.85405637  0.84033855
  0.86042311  0.8447567   0.85509014  0.85469203  0.83782005  0.83985646
  0.83986789  0.85517286  0.85630668  0.84224093  0.87072171  0.83996446
  0.87007001  0.85957952  0.83980845  0.84240615  0.85245393  0.8365153
  0.83936188  0.83977716  0.84041222  0.84112326  0.85443449  0.87592062
  0.84522527  0.8523003   0.85541887  0.8519779   0.83955101  0.87777724
  0.86509031  0.84700284  0.83954059  0.84053144  0.85550403  0.87447428
 

------------------------
100
Average: 0.815805357227
[ 0.84227235  0.9078199   0.81468615  0.87460345  0.81479296  0.81468615
  0.83892413  0.86983706  0.85733668  0.83472763  0.85632289  0.81517995
  0.8149017   0.84003871  0.81560684  0.85133987  0.82027513  0.81468615
  0.84207761  0.84436387  0.81468615  0.84963858  0.83440341  0.82386967
  0.86077793  0.81489315  0.81498221  0.89403838  0.81468615  0.84524976
  0.81491013  0.81468615  0.81517995  0.81468615  0.8453643   0.88155212
  0.81468615  0.81636527  0.83721202  0.81468615  0.81468615  0.89834351
  0.83059766  0.81560684  0.81517995  0.83759461  0.81517995  0.81468615
  0.81652091  0.84734274  0.85196237  0.81468615  0.81501662  0.84407939
  0.84015995  0.81506063  0.81480107  0.81480107  0.81480107  0.81480107
  0.82652829  0.86090731  0.81580644  0.83721009  0.82758931  0.89527127
  0.81480107  0.93283828  0.81480107  0.81509797  0.86522878  0.83173417
  0.81946124  0.84862225  0.81480107  0.82057475  0.81480107  0.8702067

KeyboardInterrupt: 

In [25]:
testEvolvedNet = testWorld.networks[0]
testEvolvedNet.printAllEdges()

testEvolvedNet.forwardStep(np.array([1.0, 0.0, 1.0, 0.0, 1.0]))

3
Edge: 3 -> 13, Enabled: True, InnovID: 54713, Weight: 1
Edge: 3 -> 21, Enabled: True, InnovID: 64926, Weight: 0.38776183774379935
Edge: 3 -> 10, Enabled: True, InnovID: 88121, Weight: 0.1320391617431315
Edge: 3 -> 24, Enabled: True, InnovID: 93091, Weight: 0.796523628987796
16
Edge: 16 -> 17, Enabled: True, InnovID: 80932, Weight: 0.42975971029392823
20
Edge: 20 -> 6, Enabled: True, InnovID: 53229, Weight: 0.8518912134873109
Edge: 20 -> 17, Enabled: True, InnovID: 66134, Weight: 0.4927472481071168
Edge: 20 -> 8, Enabled: True, InnovID: 70957, Weight: 0.40266876393525375
Edge: 20 -> 10, Enabled: True, InnovID: 88555, Weight: 0
13
Edge: 13 -> 5, Enabled: True, InnovID: 52971, Weight: 0.9505119624727593
Edge: 13 -> 7, Enabled: True, InnovID: 60041, Weight: 0.6591835830446061
18
4
Edge: 4 -> 20, Enabled: True, InnovID: 55400, Weight: 1
Edge: 4 -> 21, Enabled: True, InnovID: 61196, Weight: 0.37149917985149794
Edge: 4 -> 15, Enabled: True, InnovID: 91739, Weight: 0.8733374152086999
10
Edge

{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.61662701981533907,
 6: 0.69242682300735836,
 7: 0.58166005011740995,
 8: 0.59473429913198395,
 9: 0.5,
 10: 0.74366194817428921,
 11: 0.78632094318982071,
 12: 0.77128403937049728,
 13: 0.5,
 14: 0.6899744811276125,
 15: 0.70543966983200201,
 16: 0.68593410292272383,
 17: 0.68225734277347838,
 18: 0.73822895591692494,
 19: 0.77536804142936755,
 20: 0.95257412682243336,
 21: 0.59182118340786782,
 22: 0.69346101271476612,
 23: 0.64796430766742164,
 24: 0.67146141811236015}